In [161]:
## Notebook to compute answers to NYC procurement challenge
#  Aug 2020 by Rafal Rakowski

# Prepare compute environment
import pandas as pd
from pandas import read_html
import numpy as np
import csv
from operator import truediv 
import datetime
from datetime import date
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [162]:
# Read in data with csv API
Procurement_data = []
with open ('Recent_Contract_Awards.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        Procurement_data.append(row)
 

In [163]:
# Procurement_data[0]

In [114]:
# df = pd.DataFrame(Procurement_data, columns = list(Procurement_data[:][0]))  
# print(df)

In [115]:
# df.info()

In [164]:
# Coerce the 'StartDate' & 'EndDate' fields to datetime format 
df['StartDate']  = df['StartDate'].astype('datetime64[ns]')
df.dropna(subset = ['StartDate'], inplace=True)

# Time boundaries
start_date = "2010-01-01"
end_date   = "2019-12-31"

# Filter data by dates
after_start_date = df["StartDate"] >= start_date 
before_end_date  = df["StartDate"] <= end_date
inbetween        = after_start_date & before_end_date
filtered_df      = df.loc[inbetween]
cleaned_df       = filtered_df 

In [165]:
# Compute total sum of contracts
cleaned_df['ContractAmount'] = pd.to_numeric(cleaned_df['ContractAmount'])
sum = cleaned_df['ContractAmount'].sum() 
print ("The total contract amount is: " + str(sum))

The total contract amount is: 207783993424.84


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [166]:
cleaned_df.where(cleaned_df['AgencyName']=='Citywide Administrative Services', other=np.nan, inplace=True)
cleaned_df.dropna(subset = ['AgencyName'], inplace=True)
vendor = cleaned_df.groupby('VendorName')
vendor_amount = vendor.agg('sum')
vendor_amount.sort_values('ContractAmount', ascending = False, inplace = True)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [167]:
# Contracts amount proportion of top 50 to all 
proportion = vendor_amount['ContractAmount'][:50].sum()/vendor_amount['ContractAmount'].sum()
print ("Total number of contracts going to 50 top vendors: " + str(proportion))
new_df = vendor_amount.reset_index() 


Total number of contracts going to 50 top vendors: 0.6361324096801438


In [168]:
vendor = cleaned_df.groupby('VendorName')
vendor_count = vendor.agg('count')
vendor_count_2 = vendor_count.reset_index() 
list_vendors = vendor_count_2['VendorName'].ix[0:5]


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [169]:
# Compute ratio of the highest contract amount to the second highest
index = []
for l in list_vendors:
    index.append(int(new_df[new_df['VendorName'] == str(l)].index.values))
#index = (int(new_df[new_df['VendorName']=='Dell Marketing LP'].index.values))
print(type(index[0]))
ContractAmount = []
contract = []
mean_ContractAmount_contract = []
for ix in index:
    ContractAmount.append(new_df['ContractAmount'][ix])
for i in range(5):
    contract.append(vendor_count_2.iloc[i].iloc[1])
mean_ContractAmount_per_contract = list(map(truediv, ContractAmount, contract))
mean_ContractAmount_per_contract.sort(reverse = True) #= sorted(mean_ContractAmount_per_contract,key=float) 
ratio = mean_ContractAmount_per_contract[0]/mean_ContractAmount_per_contract[1]
print ("The ratio of the highest contract amount to the second highest is: " + str(ratio))

<class 'int'>
The ratio of the highest contract amount to the second highest is: 1.0503014505578652


In [170]:
# Compute total expenditures awarded to the ones obtained by NY vendors

# Time boundaries
start_date = "2018-01-01"
end_date = "2018-12-31"
# Filter data by dates
after_start_date = cleaned_df["StartDate"] >= start_date 
before_end_date  = cleaned_df["StartDate"] <= end_date
inbetween        = after_start_date & before_end_date
filtered_df   = cleaned_df.loc[inbetween]
cleaned_expenditure_df = filtered_df 
url = 'https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm'
NYC_zip = pd.io.html.read_html(url)
NYC_zip_df = NYC_zip[0]
split_list =[]
zip_list =[]

for row in NYC_zip_df['ZIP Codes']:
    split_list=row.split(',')
    for zip in split_list:
        zip_list.append(zip)

var = []
for zipped in cleaned_expenditure_df['VendorAddress'].str.split(',')[:]:  
    var.append(zipped[-1])  
          
bla_M = [False] * len(cleaned_expenditure_df['VendorAddress'])
result_M = np.array(bla_M)

for zip in zip_list:
    matched_zip_df = cleaned_expenditure_df['VendorAddress'].str.contains(r'\b{}\b'.format(zip))
    matched_zip_df = np.array(matched_zip_df)
    y=any(matched_zip_df.tolist())
    if  y:
        result_M = matched_zip_df | result_M
        
cleaned_expenditure_df['VendorAddress'] = result_M   
NYC_filtered = cleaned_expenditure_df.where(cleaned_expenditure_df['VendorAddress'] == True, other=np.nan)
total_award = total_expenditure['ContractAmount'][:].sum()
NYC_award = NYC_filtered ['ContractAmount'][:].sum()
outside_NYC = total_award - NYC_award 
 
print(outside_NYC)    
print(total_award)
print(NYC_award)
award_proportion = total_award/NYC_award
print(award_proportion)
print ("The ratio of total expenditures awarded to the ones obtained by NY vendors: " + str(award_proportion))

7907324507.759999
8144287025.61
236962517.85000002
34.369515902786894
The ratio of total expenditures awarded to the ones obtained by NY vendors: 34.369515902786894


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
